# マルチタスク模倣学習における表現学習のメリット

参考
* [Provable Representation Learning for Imitation Learning via Bi-level Optimization](https://arxiv.org/abs/2002.10544)

現実世界ではマルチタスク学習をしないといけないアプリケーションがたくさんありますが，MDPにおけるマルチタスクの理論はそんなに多くありません．
マルチタスクな問題設定の基本と，マルチタスク模倣学習のサンプル効率について見てみます．

MDPの表記
* time-homogeneousなMDP：$\mathcal{M}=(\mathcal{S}, \mathcal{A}, P, C, \nu)$．$C$はコスト，$\nu$は初期状態分布
* 価値関数と期待コスト：$V_h^\pi\left(s_h\right)=\mathbb{E}\left[\sum_{i=h}^H C\left(s_i, a_i\right) \mid a_i \sim \pi_i\left(s_i\right), s_{i+1} \sim P_{s_i, a_i}\right]$，$J(\boldsymbol{\pi})=\mathbb{E}_{s_1 \sim \nu} V_1^{\boldsymbol{\pi}}\left(s_1\right)$
* タスクとその分布：$\mu \sim \eta$
    * そのMDP（遷移確率は固定）：$\mathcal{M}_\mu=\left(\mathcal{S}, \mathcal{A}, H, P, C_\mu, \nu_\mu\right)$
* $\boldsymbol{\pi}_\mu^*=\left(\pi_{1, \mu}^*, \ldots, \pi_{H, \mu}^*\right)$：エキスパートの方策
* $\nu_h^\pi$：$\pi$によって誘引される$h$ステップ目での$S$上の分布．
    * その平均：$\nu_\mu^\pi=\frac{1}{H} \sum_{h=1}^H \nu_{h, \mu}^\pi$


表現学習の表記
* 方策の集合：$\Pi=\mathcal{F} \circ \Phi$
    * 状態から表現にマップする関数の集合：$\Phi \subseteq\left\{\phi: \mathcal{S} \rightarrow \mathbb{R}^d \mid\|\phi(s)\|_2 \leq R\right\}$
    * 表現から方策にマップする関数の集合：$\mathcal{F} \subseteq\left\{f: \mathbb{R}^d \rightarrow \Delta(\mathcal{A})\right\}$
    * 今回は$\mathcal{F}=\left\{x \rightarrow \operatorname{softmax}(W x) \mid W \in \mathbb{R}^{K \times d},\|W\|_F \leq 1\right\}$の形式
    * $\phi$と$f$を使って表現された方策：$\pi^{\phi, f}(a \mid s)=f(\phi(s))_a$
    * $\phi$を使って表現される方策の集合：$\Pi^\phi=\left\{\pi^{\phi, f}: f \in \mathcal{F}\right\}$




## マルチタスク模倣学習

---

**各タスクでやること**

マルチタスクになっても，通常の模倣学習とやることはあまり変わりません．
ロジスティック損失を$\ell(\pi(s), a)=-\log \left(\pi(s)_a\right)$として，各タスクについて，方策$\pi$の損失を

$$
\ell^\mu(\pi)=\underset{s \sim \nu_\mu^*, a \sim \pi_\mu^*(s)}{\mathbb{E}} \ell(\pi(s), a)
$$

とします（つまり，$\pi(s)$があまり$a$を出さない場合は損失になります）．
この差が小さい場合に $J_\mu(\pi)-J_\mu(\pi^*)$が小さくなるのは，古典的な結果から明らかですね（[RL_imitation_learning](RL_imitation_learning.ipynb)参照）

---

**タスク全体でやること**

タスク全体を通して，全部のタスクで利用できるいい感じの表現がほしいわけですね．
今回は特に$\phi$をタスク全体のサンプルから学習する状況を考えます．

* $T$個のタスク$\mu^{(1)}, \ldots, \mu^{(T)}$について，データセット $\mathbf{X}=\left\{\mathbf{x}^{(1)}, \ldots, \mathbf{x}^{(T)}\right\}$, where $\mathbf{x}^{(t)}=\left\{\left(s_j^t, a_j^t\right)\right\}_{j=1}^n \sim\left(\mu^{(t)}\right)^n$を考える

このとき，次の損失を最小化して$\hat{\phi}=\arg \min _{\phi \in \Phi} \hat{L}(\phi)$を学習します．

$$
\hat{L}(\phi):=\frac{1}{T} \sum_{t=1}^T \min _{\pi \in \Pi^\phi} \frac{1}{n} \sum_{j=1}^n \ell\left(\pi\left(s_j^t\right), a_j^t\right)
$$

---

**新しいタスクでやること**

タスク全体を通して$\hat{\phi}$を学習したので，それを使って新しいタスク$\mu\sim \eta$についての方策を学習する状況を考えます．
$\mu$からの$n$個のサンプル$x=\left\{\left(s_j, a_j\right)\right\}_{j=1}^n \sim\left(\mu\right)^n$
を使って，

$$
\pi^{\hat{\phi}, x} = \min _{\pi \in \Pi^{\hat{\phi}}} \frac{1}{n} \sum_{j=1}^n \ell\left(\pi\left(s_j\right), a_j\right)
$$

とします．
このときに，$J_\mu(\pi^{\hat{\phi}, x}) - J_\mu(\pi^*_\mu)$は小さく抑えられるでしょうか？


## 表現学習の力

任意の$\mu \in \operatorname{support}(\eta)$について，次の仮定を置きます

* $\pi^*_\mu$は決定的．つまり，$\pi^*_\mu$は何らかのコストについての最適方策．
* $a^*\arg\max_a \pi^*_\mu(a|s)$とします．$\gamma < 1/2$について，$\pi_\mu(a^*|s) \geq 1-\gamma \; \forall s \in S$を満たす$\pi_\mu \in \Pi^{\phi^*}$が存在するような$\phi^*\in \Phi$が存在するとします．つまり，全てのエキスパート方策をいい感じに表現できる表現があるとします．

このとき，次が成立します．

$\hat{\phi} \in \arg \min _{\phi \in \Phi} \hat{L}(\phi)$とします．
$T \geq c_1 \max \left\{\frac{H^4 R^2 \log (|\Phi|)}{\epsilon^2}, \frac{H^4 \ln (4 / \delta)}{\epsilon^2}\right\}$かつ$n \geq c_2 \frac{H^4 R^2 |A|}{\epsilon^2}$のとき，確率$1-\delta$以上で，

$$
\underset{\mu \sim \eta \mathbf{x} \sim \mu^n}{\mathbb{E}} J_\mu\left(\pi^{\hat{\phi}, \mathbf{x}}\right)-\underset{\mu \sim \eta}{\mathbb{E}} J_\mu\left(\pi_\mu^*\right) \leq H^2 \gamma+\epsilon
$$

が成立します．

各タスクを独立して学習させると$n=O\left(\max \left\{\frac{H^4 R^2 \log (|\Phi|)}{\epsilon^2}, \frac{H^4 R^2 |A|}{\epsilon^2}\right\}\right)$のサンプルが必要になるので，$\log (|\Phi|) \gg |A|$の場合には，このマルチタスク学習のフレームワークはサンプル効率が良いことになります．